Geospatial analysis and interactive mapping of SpaceX launch site locations with Folium

In [ ]:
# !pip install folium
# !pip install wget
# !pip install pandas

In [1]:
import folium
import pandas as pd

In [2]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [3]:
#Import SpaceX launch site locations data
spacex_df = pd.read_csv('data/spacex_launch_geo.csv')

In [5]:
#Keep only necessary columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
#Loations of four launch sites
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [10]:
#Set the initial location of the map to be Spaceport
spaceport_coordinate = [32.990254, -106.974998] 
site_map = folium.Map(location=spaceport_coordinate, zoom_start=10)

In [12]:
#Add a circle marker with a popup to highlight Spaceport on the map
circle = folium.Circle(spaceport_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('Spaceport'))
#Add a custom marker with a text label to represent Spaceport on the map
marker = folium.map.Marker(
    spaceport_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'Spaceport',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [13]:
# Initialise the map
site_map = folium.Map(location=spaceport_coordinate, zoom_start=5)

#Coordinates of launch sites
ccafs_lc = [28.56230197, -80.57735648]
ccafs_slc = [28.56319718, -80.57682003]
klc_lc = [28.57325457, -80.64689529]
vafb_slc = [34.63283416, -120.6107455]

#Add a marker and a label in each launch site
circle_ccafs_lc = folium.Circle(ccafs_lc, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('CCAFS LC-40'))
circle_ccafs_slc = folium.Circle(ccafs_slc, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('CCAFS SLC-40'))
circle_klc_lc = folium.Circle(klc_lc, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('KSC LC-39A'))
circle_vafb_slc = folium.Circle(vafb_slc, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('VAFB SLC-4E'))

for circle in [circle_ccafs_lc, circle_ccafs_slc, circle_klc_lc, circle_vafb_slc]:
    site_map.add_child(circle)

marker_ccafs_lc = folium.map.Marker(ccafs_lc, icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS LC-40',
    ))

marker_ccafs_slc = folium.map.Marker(ccafs_slc, icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS SLC-40',
    ))

marker_klc_lc = folium.map.Marker(klc_lc, icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'KSC LC-39A',
    ))

marker_vafb_slc = folium.map.Marker(vafb_slc, icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'VAFB SLC-4E',
    ))

for marker in [marker_ccafs_lc, marker_ccafs_slc, marker_klc_lc, marker_vafb_slc]:
    site_map.add_child(marker)

site_map

In [16]:
#Create a marker cluster to group multiple markers
marker_cluster = MarkerCluster()

In [14]:
#Function to assign marker colors based on launch outcome, green for success and red for failure
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [17]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

#Add markers for each SpaceX launch site to the marker cluster with colors based on their outcome
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
    location=[record['Lat'], record['Long']],
    icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

In [18]:
#Add a mouse position control to display latitude and longitude on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [20]:
from math import sin, cos, sqrt, atan2, radians

#Function to calculate the distance between two geographic coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [21]:
#Caluclate the distance between VAFB SLC-4E launch site and the neaerest coastline 
launch_site_lat = 34.63283416
launch_site_long = -120.6107455
coastline_lat = 34.63625
coastline_long = -120.62495
distance_coastline = calculate_distance(launch_site_lat, launch_site_long, coastline_lat, coastline_long)

In [22]:
#Add a marker to display the distance to the coastline in kilometers on the map
coastline = [34.63625, -120.62495]
distance_marker = folium.Marker(
   coastline,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )
site_map.add_child(distance_marker)

In [23]:
#Add a polyline to the map to represent the distance between the launch site and the coastline
lines=folium.PolyLine(locations=[vafb_slc, coastline], weight=2)
site_map.add_child(lines)

In [24]:
#Add a marker and polyline to display the distance between the launch site and the nearest city

city_lat = 34.63801
city_long = -120.45856
distance_city = calculate_distance(launch_site_lat, launch_site_long, city_lat, city_long)

city = [34.63801, -120.45856]
distance_marker_city = folium.Marker(
   city,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
       )
   )
site_map.add_child(distance_marker_city)

line_city=folium.PolyLine(locations=[vafb_slc, city], weight=2)
site_map.add_child(line_city)

In [25]:
##Add a marker and polyline to display the distance between the launch site and the nearest railway 
railway_lat = 34.63709
railway_long = -120.62345
distance_railway = calculate_distance(launch_site_lat, launch_site_long, railway_lat, railway_long)

railway = [34.63709, -120.62345]
distance_marker_railway = folium.Marker(
   railway,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
       )
   )
site_map.add_child(distance_marker_railway)

line_railway=folium.PolyLine(locations=[vafb_slc, railway], weight=2)
site_map.add_child(line_railway)

In [26]:
##Add a marker and polyline to display the distance between the launch site and the nearest highway 
highway_lat = 34.68519
highway_long = -120.45942
distance_highway = calculate_distance(launch_site_lat, launch_site_long, highway_lat, highway_long)

highway = [34.68519, -120.45942]
distance_marker_highway = folium.Marker(
   highway,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
       )
   )
site_map.add_child(distance_marker_highway)

line_highway=folium.PolyLine(locations=[vafb_slc, highway], weight=2)
site_map.add_child(line_highway)

site_map

Launch sites are located in either the west or east coast of the United States and they are all close to the sea.
Among launches from the VAFB SLC-4E launch sites, 4 of them were successful (marked in green) and the rest was unsuccessful (marked in red).
VAFB SLC-4E launch site is close to a coastline and a railway, but relatively far from a city and a highway.
